<a href="https://colab.research.google.com/github/DenisKai7/invoice_generator/blob/main/malam_puncak.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
# Install library yang diperlukan
!pip install gspread pandas fpdf2 pyqrcode pypng requests pillow
!pip install --upgrade google-auth-oauthlib google-auth-httplib2

# Import library yang dibutuhkan
import gspread
from google.colab import auth
from google.auth import default
import pandas as pd
from fpdf import FPDF
import os
from google.colab import drive
from datetime import datetime
import requests
from io import BytesIO
from PIL import Image
import urllib.parse
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload

# Autentikasi Google Colab dengan Google Sheets & Drive
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# Inisialisasi Google Drive API
drive_service = build('drive', 'v3', credentials=creds)

# Mount Google Drive untuk menyimpan invoice
drive.mount('/content/drive')

# Buka spreadsheet
spreadsheet_url = "https://docs.google.com/spreadsheets/d/1Eig9wEouZRPDxJvmdvNBJ8g2XYAaXGFPS-bwDtVXcK8/edit?gid=0#gid=0"
sh = gc.open_by_url(spreadsheet_url)
worksheet = sh.get_worksheet(0)

# Ambil semua data dan periksa kolom yang tersedia
expected_headers = ['KODE INVOICE', 'Tanggal', 'email', 'KATEGORI', 'NAMA', 'NO WA', 'KODE REFERAL', 'Quantity', 'Jenis Tiket', 'Barcode', 'template_pesan']
records = worksheet.get_all_records(expected_headers=expected_headers)
df = pd.DataFrame.from_records(records)

# Periksa nama kolom yang sebenarnya
print("Kolom yang tersedia di spreadsheet:")
print(df.columns.tolist())

# Pastikan nama kolom sesuai dengan spreadsheet Anda
COLUMN_MAPPING = {
    'invoice_id': 'KODE INVOICE',
    'date': 'Tanggal',
    'email': 'email',
    'category': 'KATEGORI',
    'name': 'NAMA',
    'whatsapp': 'NO WA',
    'referral_code': 'KODE REFERAL',
    'quantity': 'Quantity',
    'ticket' : 'Jenis Tiket',
    'barcode': 'Barcode',
    'message_template': 'template_pesan'
}

# Path logo di Google Drive (sesuaikan dengan path Anda)
LOGO_DIES_PATH = '/content/drive/MyDrive/logo/diesnatalis 1-01.png'  # Ganti dengan path yang benar
LOGO_UNIPMA_PATH = '/content/drive/MyDrive/logo/Unipma.png'  # Ganti dengan path yang benar

class InvoicePDF(FPDF):
    def __init__(self, participant_data):
        super().__init__(format='A4', orientation='L')  # A4 Landscape
        self.participant_data = participant_data
        self.add_page()

    def header(self):
        # Simpan posisi Y awal
        start_y = self.get_y()

        # Tambahkan background light grey
        self.set_fill_color(220, 220, 220)  # Light grey (RGB)
        self.rect(0, 0, self.w, 60, 'F')  # Rectangle selebar halaman dengan tinggi 60

        # Tambahkan background dengan efek fade di atas light grey
        self._add_faded_background()

        # Tambahkan logo jika file ada
        try:
            # Logo Dies Natalis di kiri - ukuran disesuaikan
            if os.path.exists(LOGO_UNIPMA_PATH):
                self.image(LOGO_UNIPMA_PATH, x=15, y=15, w=30)  # Ukuran lebih besar untuk A4
        except Exception as e:
            print(f"⚠️ Error loading UNIPMA logo: {e}")

        try:
            # Logo UNIPMA di kanan - ukuran disesuaikan
            if os.path.exists(LOGO_DIES_PATH):
                self.image(LOGO_DIES_PATH, x=210, y=0, w=100)  # Ukuran lebih besar untuk A4
        except Exception as e:
            print(f"⚠️ Error loading Dies Natalis logo: {e}")

        # Judul
        self.set_y(15)
        self.set_font('Arial', 'B', 18)
        self.cell(0, 10, 'GROOVE SPECTA 2025', 0, 1, 'C')
        self.set_font('Arial', 'I', 12)
        self.cell(0, 7, 'Official Invoice', 0, 1, 'C')
        self.set_font('Arial', 'B', 11)
        self.cell(0, 7, '02 Juni 2025', 0, 1, 'C')
        self.set_font('Arial', 'I', 11)
        self.cell(0, 7, 'Jl. Letkol Suwarno No. 15, Kanigoro, Kec. Kartoharjo, Kota Madiun', 0, 1, 'C')
        self.ln(5)

    def _add_faded_background(self):
        # Path ke gambar background (sesuaikan dengan path Anda)
        BG_IMAGE_PATH = '/content/drive/MyDrive/logo/gs.png'  # Ganti dengan path gambar background Anda

        if os.path.exists(BG_IMAGE_PATH):
            try:
                # Simpan posisi Y saat ini
                current_y = self.get_y()

                # Tentukan tinggi header (sesuaikan dengan tinggi header Anda)
                header_height = 60  # Contoh: 60 unit tinggi untuk header

                x_offset = 0  # Sesuaikan nilai ini untuk mengatur seberapa jauh ke kiri
                adjusted_width = self.w + abs(x_offset)  # Tambahkan lebar untuk kompensasi pergeseran

                # Tambahkan gambar background dengan opacity rendah HANYA di area body (di bawah header)
                self.image(BG_IMAGE_PATH, x=x_offset, y=header_height, w=adjusted_width, h=self.h-header_height)

                # Tambahkan layer semi-transparan untuk efek fade HANYA di area body
                self.set_fill_color(255, 255, 255, 200)  # Putih dengan alpha 200/255 (~78% opacity)
                self.rect(0, header_height, self.w, self.h-header_height, style='F')

                # Kembalikan posisi Y
                self.set_y(current_y)
            except Exception as e:
                print(f"⚠️ Error adding background: {e}")
        else:
            print("⚠️ Background image not found, skipping...")

    def footer(self):
        # Thank you message di bagian footer tengah
        self.set_y(-15)
        self.set_font('Arial', 'I', 12)
        self.cell(0, 10, 'System By : Jofanza Denis Aldida & Yoga Gondrong', 0, 0, 'C')

    def create_invoice(self):
        # Informasi invoice
        self.set_font('Arial', 'B', 14)
        self.set_y(60)  # Atur posisi Y secara eksplisit (75mm dari atas)
        self.cell(0, 10, f'INVOICE #{self.participant_data[COLUMN_MAPPING["invoice_id"]]}', 0, 1, 'L')

        # Tanggal
        self.set_font('Arial', '', 11)
        self.cell(0, 8, f'Date: {datetime.now().strftime("%d %B %Y")}', 0, 1, 'L')
        self.ln(3)

        # Garis pemisah
        self.line(10, self.get_y(), 287, self.get_y())
        self.ln(4)

        # Layout 3 kolom dengan alignment yang presisi
        col_width = 85
        start_y = self.get_y()  # Simpan posisi Y awal

        # Kolom 1: Data peserta dalam bentuk tabel transparan
        self.set_font('Arial', 'B', 12)
        self.cell(0, 8, 'PARTICIPANT DETAILS', 0, 1, 'L')
        self.ln(3)

        # Warna palette modern transparan
        text_color = (50, 50, 50)  # Dark grey text
        border_color = (200, 200, 200)  # Light grey border

        # Simpan posisi Y untuk alignment
        y_details = self.get_y()

        # Data yang akan ditampilkan (semua field wajib)
        participant_info = [
            ("Nama Lengkap", str(self.participant_data[COLUMN_MAPPING["name"]])),
            ("Nomor HP", str(self.participant_data[COLUMN_MAPPING["whatsapp"]])),
            ("Kategori", str(self.participant_data[COLUMN_MAPPING["category"]])),
            ("Kode Referral", str(self.participant_data[COLUMN_MAPPING["referral_code"]]) if self.participant_data[COLUMN_MAPPING["referral_code"]] else '-')
        ]

        # Lebar kolom
        label_width = 40
        value_width = 60

        # Buat tabel tanpa background fill (transparan)
        self.set_draw_color(*border_color)  # Warna border light grey
        self.set_line_width(0.3)  # Garis tipis

        # Header tabel (transparan)
        self.set_font('Arial', 'B', 10)
        self.set_text_color(*text_color)
        self.cell(label_width, 8, 'INFORMASI', 'B', 0, 'L')
        self.cell(value_width, 8, 'DETAIL', 'B', 1, 'L')

        # Isi tabel (transparan)
        for label, value in participant_info:
            # Label
            self.set_font('Arial', 'B', 9)
            self.cell(label_width, 8, label, 'B', 0, 'L')

            # Value
            self.set_font('Arial', '', 9)
            self.cell(value_width, 8, value, 'B', 1, 'L')

        # Simpan tinggi tabel participant details
        participant_details_height = self.get_y() - y_details
        self.ln(5)  # Spasi setelah tabel

        # Kolom 2: Detail ticket (tengah) - sejajar dengan participant details
        self.set_xy(120, y_details - 5)  # Sejajar dengan "PARTICIPANT DETAILS"
        self.set_font('Arial', 'B', 12)
        self.cell(col_width, 8, 'TICKET DETAILS', 0, 1, 'L')
        self.ln(3)

        # Data yang akan ditampilkan dari spreadsheet
        ticket_info = [
            ("Quantity", str(self.participant_data[COLUMN_MAPPING["quantity"]])),
            ("Jenis Tiket", str(self.participant_data[COLUMN_MAPPING["ticket"]])),
            ("Paid On", str(self.participant_data[COLUMN_MAPPING["date"]]))
        ]

        # Buat tabel tanpa background fill (transparan) sama seperti participant details
        self.set_draw_color(*border_color)  # Warna border light grey
        self.set_line_width(0.3)  # Garis tipis

        # Header tabel (transparan)
        self.set_font('Arial', 'B', 10)
        self.set_text_color(*text_color)
        self.cell(label_width, 8, 'INFORMASI', 'B', 0, 'L')
        self.cell(value_width, 8, 'DETAIL', 'B', 1, 'L')

        # Isi tabel (transparan)
        for label, value in ticket_info:
            # Label
            self.set_font('Arial', 'B', 9)
            self.cell(label_width, 8, label, 'B', 0, 'L')

            # Value
            self.set_font('Arial', '', 9)
            self.cell(value_width, 8, value, 'B', 1, 'L')

        # Simpan posisi Y setelah ticket details
        ticket_details_end_y = self.get_y()

        self.set_xy(10, ticket_details_end_y + 10)

        # Tambahkan informasi event details di bawah tabel
        self.set_font('Arial', 'B', 12)
        self.cell(0, 8, 'PENUKARAN TIKET', 0, 1, 'C')
        self.ln(3)

        # Detail penukaran tiket
        exchange_info = [
            "31 Mei 2025  : 10.00-16.00 WIB",
            "01 Juni 2025 : 10.00-16.00",
            "02 Juni 2025 : 08.00-16.00",
            "Lokasi: Area Gor Cendekia"
        ]

        self.set_font('Arial', '', 10)
        for info in exchange_info:
            self.cell(0, 7, info, 0, 1, 'C')

        # Kolom 3: Barcode/QR code - sejajar dengan participant details
        if COLUMN_MAPPING["barcode"] in self.participant_data and self.participant_data[COLUMN_MAPPING["barcode"]]:
            try:
                response = requests.get(self.participant_data[COLUMN_MAPPING["barcode"]])
                img = Image.open(BytesIO(response.content))

                temp_img_path = f"/tmp/barcode_{self.participant_data[COLUMN_MAPPING['invoice_id']]}.png"
                img.save(temp_img_path)

                # Posisi di kolom kanan, sejajar dengan participant details
                x_position = 220
                y_position = y_details - 5  # Sejajar dengan "PARTICIPANT DETAILS"

                # Tambahkan judul barcode
                self.set_xy(x_position, y_position)
                self.set_font('Arial', 'B', 12)
                self.cell(60, 8, 'Barcode Ticket:', 0, 1, 'L')

                # Tambahkan gambar barcode
                self.image(temp_img_path, x=x_position, y=y_position + 10, w=60)

                # Teks di bawah barcode
                self.set_xy(x_position, y_position + 70)
                self.set_font('Arial', 'I', 10)
                self.cell(60, 6, 'Scan this barcode for verification', 0, 1, 'C')

                # Teks di bawah barcode 2
                self.set_xy(x_position, y_position + 76)
                self.set_font('Arial', 'I', 10)
                self.cell(60, 6, 'Thank you for joining our event!', 0, 1, 'C')

                os.remove(temp_img_path)
            except Exception as e:
                print(f"⚠️ Error adding barcode: {e}")

        # Notes di bagian bawah - diposisikan lebih tinggi
        notes_y = max(self.get_y() - 10, 120)  # Mengurangi 15pt dari posisi sebelumnya
        self.set_xy(10, notes_y)
        self.set_font('Arial', 'I', 11)
        notes = [
            "Catatan:",
            "- Harap datang tepat waktu sesuai sesi yang dipilih",
            "- Bawa identitas yang valid (KTP/SIM/KTM/KARTU PELAJAR)"
        ]
        for note in notes:
            self.cell(0, 6, note, ln=1, align='L')

def generate_invoice_pdf(participant_data):
    # Buat PDF
    pdf = InvoicePDF(participant_data)
    pdf.create_invoice()

    # Buat folder penyimpanan di Colab
    local_invoice_dir = "/content/invoices"
    os.makedirs(local_invoice_dir, exist_ok=True)

    # Simpan PDF lokal
    pdf_filename = f"GROOVE SPECTA_{participant_data[COLUMN_MAPPING['invoice_id']]}.pdf"
    local_pdf_path = f"{local_invoice_dir}/{pdf_filename}"
    pdf.output(local_pdf_path)

    return local_pdf_path, pdf_filename

def upload_to_drive(file_path, file_name, folder_id=None):
    """Upload file ke Google Drive dan set permissions"""
    file_metadata = {
        'name': file_name,
        'parents': [folder_id] if folder_id else None
    }

    media = MediaFileUpload(file_path, mimetype='application/pdf')

    file = drive_service.files().create(
        body=file_metadata,
        media_body=media,
        fields='id,webViewLink'
    ).execute()

    # Set permissions agar bisa diakses oleh siapa saja dengan link
    permission = {
        'type': 'anyone',
        'role': 'reader'
    }

    drive_service.permissions().create(
        fileId=file['id'],
        body=permission
    ).execute()

    return file['webViewLink']

def generate_message_template(participant_data, drive_link):
    # Format WhatsApp number dengan +62
    whatsapp_num = str(participant_data.get(COLUMN_MAPPING['whatsapp'], ''))
    if whatsapp_num:
        clean_number = ''.join(filter(str.isdigit, whatsapp_num))
        if clean_number.startswith('0'):
            clean_number = '+62' + clean_number[1:]
        elif not clean_number.startswith('62'):
            clean_number = '+62' + clean_number
        else:
            clean_number = '+' + clean_number
    else:
        clean_number = ''

    return f"""
Halo {participant_data[COLUMN_MAPPING['name']]},

Terima kasih telah mendaftar di *GROOVE SPECTA 2025*!
Berikut invoice resmi Anda:

🔹 *Kode Invoice:* {participant_data[COLUMN_MAPPING['invoice_id']]}
🔹 *Kategori:* {participant_data[COLUMN_MAPPING['category']]}

📥 *Download Invoice:*
{drive_link}

*Catatan:*
- Harap datang tepat waktu sesuai sesi yang dipilih
- Bawa identitas yang valid (KTP/SIM/KTM/KARTU PELAJAR)

Mohon simpan invoice ini sebagai bukti pendaftaran.
Jika ada pertanyaan, hubungi kami via WhatsApp.

Salam,
*Panitia GROOVE SPECTA 2025*
"""

def process_all_participants():
    updates = []
    errors = []

    # ID folder tujuan di Google Drive (ganti dengan folder ID Anda)
    DRIVE_FOLDER_ID = '1TgkYlvaWh_Wy5heW3zaP6b6WDMbGsxKG'  # Ganti dengan folder ID tujuan

    for index, row in df.iterrows():
        try:
            invoice_id = row[COLUMN_MAPPING['invoice_id']]
            print(f"\nMemproses peserta: {row.get(COLUMN_MAPPING['name'], '')} (Invoice: {invoice_id})")

            # 1. Generate PDF
            local_pdf_path, pdf_filename = generate_invoice_pdf(row)

            # 2. Upload ke Google Drive dan dapatkan link
            drive_link = upload_to_drive(local_pdf_path, pdf_filename, DRIVE_FOLDER_ID)
            print(f"✅ Invoice diupload ke: {drive_link}")

            # 3. Buat template pesan
            message = generate_message_template(row, drive_link)
            updates.append((index, 'template_pesan', message))

            # 4. Bersihkan file lokal
            os.remove(local_pdf_path)

        except Exception as e:
            error_msg = f"❌ Error saat memproses {row.get(COLUMN_MAPPING['name'], '')}: {str(e)}"
            print(error_msg)
            errors.append((index, error_msg))

    # Update spreadsheet
    for update in updates:
        row_idx, col_name, value = update
        try:
            # Cari kolom yang sesuai
            col_names = [col.lower() for col in df.columns]
            target_col = col_name.lower()

            if target_col in col_names:
                col_idx = col_names.index(target_col) + 1  # +1 karena indeks spreadsheet mulai dari 1
                worksheet.update_cell(row_idx + 2, col_idx, value)
                print(f"✔️ Updated {col_name} untuk baris {row_idx + 2}")
            else:
                print(f"⚠️ Kolom {col_name} tidak ditemukan di spreadsheet")
        except Exception as e:
            print(f"❌ Gagal mengupdate spreadsheet untuk baris {row_idx}: {str(e)}")

    # Tampilkan error summary
    if errors:
        print("\n⛔ Error Summary:")
        for error in errors:
            print(error[1])

# Jalankan proses utama
process_all_participants()
print("\n✅ Proses selesai!")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Kolom yang tersedia di spreadsheet:
['KODE INVOICE', 'Tanggal', '', 'email', 'KATEGORI', 'NAMA', 'NO WA', 'KODE REFERAL', 'Quantity', 'Jenis Tiket', 'Barcode', 'template_pesan', 'Action']

Memproses peserta: MUHAMMAD RIFA'I (2) (Invoice: A0069KEO)
⚠️ Error adding background: FPDF.set_fill_color() takes from 2 to 4 positional arguments but 5 were given


<ipython-input-8-8a2a3d44c151>:99: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  self.set_font('Arial', 'B', 18)
<ipython-input-8-8a2a3d44c151>:100: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=1 use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  self.cell(0, 10, 'GROOVE SPECTA 2025', 0, 1, 'C')
<ipython-input-8-8a2a3d44c151>:101: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  self.set_font('Arial', 'I', 12)
<ipython-input-8-8a2a3d44c151>:102: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=1 use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  self.cell(0, 7, 'Official Invoice', 0, 1, 'C')
<ipython-input-8-8a2a3d44c151>:103: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  self.set_font('Arial'

⚠️ Error adding background: FPDF.set_fill_color() takes from 2 to 4 positional arguments but 5 were given
✅ Invoice diupload ke: https://drive.google.com/file/d/18UdmwpIyXCX-h-JX3mFhqKKAQrnla4Dz/view?usp=drivesdk

Memproses peserta: Putri (Invoice: A0070WYB)
⚠️ Error adding background: FPDF.set_fill_color() takes from 2 to 4 positional arguments but 5 were given
⚠️ Error adding background: FPDF.set_fill_color() takes from 2 to 4 positional arguments but 5 were given
✅ Invoice diupload ke: https://drive.google.com/file/d/1gtyhyXwCTrJ2IIbO-Mbiv0RB8E-CI9Qk/view?usp=drivesdk

Memproses peserta: RIZKI BUDI SETIAWAN(1) (Invoice: A0071WYK)
⚠️ Error adding background: FPDF.set_fill_color() takes from 2 to 4 positional arguments but 5 were given
⚠️ Error adding background: FPDF.set_fill_color() takes from 2 to 4 positional arguments but 5 were given
✅ Invoice diupload ke: https://drive.google.com/file/d/1xoHsSVgA3BOhmLPQGWMWKwfzpKqhby3u/view?usp=drivesdk

Memproses peserta: RIFDA KARUNIA MUKTI 